# **SI649 W25 Altair Theme Homework #3**

# Overview


We are focusing on **custom themes & small multiples** in this lab! For this assignment, we will be looking at *Star Wars* character dataset from [America’s Favorite ‘Star Wars’ Movies (And Least Favorite Characters)](https://fivethirtyeight.com/features/americas-favorite-star-wars-movies-and-least-favorite-characters/) by Walt Hickey.

### Lab Instructions

*   Save, rename, and submit the ipynb file (use your username in the name).
*   Complete all the checkpoints, to create the required visualization at each cell
*   Run every cell (do Runtime -> Restart and run all to make sure you have a clean working version), and upload your .ipynb file to Canvas.
*   For each visualization, there is a space to write down a "Grammar of Graphics" plan, but this is optional for this assignment.
*   If you end up stuck, show us your work by including links (URLs) that you have searched for. You'll get partial credit for showing your work in progress.


In [53]:
# suppress warnings about future deprecations
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# imports we will use
import altair as alt
import pandas as pd

# for large data sets
alt.data_transformers.disable_max_rows()

# read in data
df = pd.read_csv('https://scmcqueen.github.io/si649_hw/skyelerStarWars.csv',header=[0,1],skip_blank_lines=True)



## Part 1: Character Rating Facet Charts

Using an Altair Facet chart, recreate the 'Star Wars' Character Favorability Ratings chart from the [Star Wars article](https://fivethirtyeight.com/features/americas-favorite-star-wars-movies-and-least-favorite-characters/). It should look like this:

<img src="https://scmcqueen.github.io/si649_hw/favorability_SW.png" height="300">

### Step 1: Get data in the correct format
We will get this started for you

In [54]:
# Read in the data with Character name & review
download = pd.read_csv('https://scmcqueen.github.io/si649_hw/reviews_by_char.csv',index_col=0)
download.head()

,character,rating
0,Han Solo,Very favorably
2,Han Solo,Somewhat favorably
3,Han Solo,Very favorably
4,Han Solo,Very favorably
5,Han Solo,Very favorably


In [55]:
# TODO: Map the 'rating' column to a new column that matches the chart's Favorable, Unfavorable, etc.
download['class'] = download['rating'].map({'Very favorably':'Favorable','Somewhat favorably':'Favorable','Unfamiliar (N/A)':'Unfamiliar', 'Somewhat unfavorably':'Unfavorable','Neither favorably nor unfavorably (neutral)':'Neutral', 'Very unfavorably':'Unfavorable'})

In [56]:
# TODO: in Pandas, get the percent of each rating per character
rating_counts = download.groupby(['character', 'class']).size().reset_index(name='count')
total_counts = download.groupby(['character']).size().reset_index(name='total')
df_percent = rating_counts.merge(total_counts, on='character')
df_percent['percent'] = df_percent['count'] / df_percent['total'] * 100
favorable_sort = df_percent[df_percent['class'] == 'Favorable']
df_percent = df_percent.merge(
    favorable_sort[['character', 'percent']], 
    on='character', 
    suffixes=('', '_favorable')
)

### Step 2: Create your charts


Hints:
* Layer the charts before faceting!

In [57]:
bar1 = alt.Chart(df_percent).mark_bar().encode(
    x=alt.X('percent:Q', axis=alt.Axis(title=None, grid=False, labels=False, ticks=False, domain=False)),
    y=alt.Y('character:N', axis=alt.Axis(title=None, grid=False),sort=alt.EncodingSortField(field="percent_favorable", order="descending")),
    color=alt.Color('class:N', legend=None),
).properties(
    width=80,
)
text1 = bar1.mark_text(
    align='left',
    baseline='middle',
    dx=3,
    fontSize=10,
).encode(
    text=alt.Text('percent:Q', format=".0f")
)
(bar1 + text1).facet(
    column=alt.Column('class:N', sort=['Favorable', 'Neutral', 'Unfavorable', 'Unfamiliar'],header=alt.Header(title=None)),
).configure_view(
    stroke=None
).properties(
    title={
        "text": "'Star Wars' Favorability Rating",
        "subtitle": "By 834 respondents",
        "anchor": "start"
    }
)


alt.FacetChart(...)

## Part 2: Star Wars Theme

### Step 1: Create Star Wars Theme

We want you to try implementing a custom theme in Altair based on this style guide:

<img src="https://scmcqueen.github.io/si649_hw/StarWars_StyleGuide.png" height="600">

We will give you some starter code, so you aren't creating a theme fully from scratch. The Altair documentation on [Chart Customization](https://altair-viz.github.io/user_guide/customization.html) and this Towards Data Science Article [Consistently Beautiful Visualizations with Altair Themes](https://medium.com/towards-data-science/consistently-beautiful-visualizations-with-altair-themes-c7f9f889602) should serve as helpful guides.

You can set the spacing, color palettes, font schemes, etc.

Run the cell below to get the font you need.

In [58]:
%%html

<style>
@import url('https://fonts.googleapis.com/css?family=Lato');
</style>

In [59]:
# TO DO: Modify this code to fit the above style guide
#@alt.theme.register("star_wars", enable=True) # Comment this line out for Altair 5.2
# the theme is defined as a function
def star_wars_solution():
    font = "Lato"
    backgroundColor = None

    return {
        "config": {
            "title": {
                "anchor": "start",
                "fontSize": 20,
                "font": font,
                "color": "#000000"
            },
            "subtitle": {
                "fontSize": 18,
                "font": font,
                "color": "#000000"
            },
            "header": {
                "titleColor": "#00274C",
                "titleFontSize": 1,
                "titleFontWeight": "bold",
                "titleBaseline": "top"
            },
            "axisX": {
                "titleFont": font,
                "titleFontSize": 16,
                "titleColor": "#000000",
                "labelFont": font,
                "labelFontSize": 14,
                "labelColor": "#000000",
                "domain": True, 
                "grid": False,
                "ticks": False, 
            },
            "axisY": {
                "titleFont": font,
                "titleFontSize": 16,
                "titleColor": "#000000",
                "titleAngle": 0,
                "titleAlign": "left",
                "titleAnchor": "start",
                "titleY": -10,
                "labelFont": font,
                "labelFontSize": 14,
                "labelColor": "#000000",
                "domain": False,
                "grid": True,  
                "gridColor": "#aab0c0",
                "gridWidth": 1
            },
            "legend": {
                "labelFont": font,
                "labelFontSize": 14,
                "titleFont": font,
                "titleFontSize": 16,
            },
            "background": backgroundColor,
            "view": {
                "stroke": "transparent",
            },
            "range": {
                "category": ["#B62321", "#A1A332", "#15509F", "#3B444B", "#D7C078", "#DC5026"],
                "ordinal": ["#B62321", "#BD4232", "#C36244", "#CA8155", "#D0A167", "#D7C078"],
                "ramp": ["#BD4232", "#C36244", "#CA8155", "#D0A167", "#D7C078"],
                "diverging": ["#15509F", "#638ABF", "#B1C5DF", "#FFFFFF", "#F3C5B7", "#E88A6E", "#DC5026"]
            },
            "text": {
                "font": font,
                "fontSize": 14,
                "color": "match" 
            },
            "note": {
                "font": font,
                "fontSize": 12,
                "color": "#000000",
                "align": "left"
            },
            "source": {
                "font": font,
                "fontSize": 12,
                "color": "#000000",
                "align": "right"
            },
        }
    }

# For Altair version 5.2, uncomment the lines below
alt.themes.register('star_wars', star_wars_solution)
# # enable the newly registered theme
alt.themes.enable('star_wars')

ThemeRegistry.enable('star_wars')

### Step 2: Recreate Favorability Chart with new theme

To check that your theme looks correct, recreate your chart of Star Wars characters favorability. If your theme is enabled correctly, the visualization should look like this:

<img src="https://scmcqueen.github.io/si649_hw/favorability_SW_theme.png" height="300">

In [60]:
bar1 = alt.Chart(df_percent).mark_bar().encode(
    x=alt.X('percent:Q', axis=alt.Axis(title=None, grid=False, labels=False, ticks=False, domain=False)),
    y=alt.Y('character:N', axis=alt.Axis(title=None, grid=False),sort=alt.EncodingSortField(field="percent_favorable", order="descending")),
    color=alt.Color('class:N', legend=None),
).properties(
    width=80,
)
text1 = bar1.mark_text(
    align='left',
    baseline='middle',
    dx=3,
    fontSize=10,
).encode(
    text=alt.Text('percent:Q', format=".0f")
)
(bar1 + text1).facet(
    column=alt.Column('class:N', sort=['Favorable', 'Neutral', 'Unfavorable', 'Unfamiliar'],header=alt.Header(title=None)),
).configure_view(
    stroke=None
).properties(
    title={
        "text": "'Star Wars' Favorability Rating",
        "subtitle": "By 834 respondents",
        "anchor": "start"
    }
)


alt.FacetChart(...)

### Step 3: Recreate the following charts to test your theme

#### Chart 1: Favorable Ratings by Number of Ratings

<img src="https://scmcqueen.github.io/si649_hw/favorable.png" height="300">

In [61]:
# TODO: Create favorability chart
df_c1 = df_percent[df_percent['class'] == 'Favorable']
df_c1['c1'] =  df_c1['percent']/100


/var/folders/gp/bnf8n57s1nlgccs8xl1kk3mc0000gn/T/ipykernel_93959/999980883.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_c1['c1'] =  df_c1['percent']/100


In [62]:
alt.Chart(df_c1).mark_point(filled=True, size=50).encode(
    x=alt.X('total:Q', scale=alt.Scale(domain=[800, 850]), axis=alt.Axis(title='Number of Ratings')),
    y=alt.Y('c1:Q', axis=alt.Axis(title='Percent of Favorable Ratings', tickCount=6)),
    color=alt.Color('c1:Q', title='Percent'),
).properties(
    title='Favorable Ratings by Number of Ratings',
    width=300,
    height=300
)

alt.Chart(...)

#### Chart 2: Who shot first?

<img src="https://scmcqueen.github.io/si649_hw/shot_first.png" height="150">

In [63]:
# read in shooting data
shot_first = pd.read_csv('https://scmcqueen.github.io/si649_hw/shot_first.csv',index_col=0)
shot_first.head()

,shooter
0,I don't understand this question
2,I don't understand this question
3,I don't understand this question
4,Greedo
5,Han


In [64]:
df_count = shot_first['shooter'].value_counts().reset_index()
df_count.columns = ['shooter', 'count']
total = df_count['count'].sum()
df_count['percent'] = df_count['count'] / total
order_list = ['Greedo', 'Han', "I don't understand this question"]
df_count['shooter'] = pd.Categorical(df_count['shooter'], categories=order_list, ordered=True)
df_count.sort_values('shooter', inplace=True)

In [90]:
bar = (
    alt.Chart(df_count)
    .mark_bar(height=30)
    .encode(
        y=alt.Y('shooter:N', sort=None, title='', axis=alt.Axis(grid=False, ticks=False)),
        x=alt.X('percent:Q',
                title=' ',
                axis=alt.Axis(title=None, grid=False, labels=False, ticks=False, domain=False),),
        color=alt.Color('percent:Q', title='Percent of Responders',
                        scale=alt.Scale(domain=[0.2, 0.3, 0.4], range=["#15509F", "#FFFFFF", "#DC5026"] ))
    ).properties(
        title='Who shot first?',
        width=300,
        height=100
    )
)
text = bar.mark_text(
    align='right',
    baseline='middle',
    dx=-3,
    color='white'
).encode(
    text=alt.Text('percent:Q', format='.2%'),
    color=alt.value('white')
)

final_chart = (bar + text)
final_chart



alt.LayerChart(...)

## Part 3: Repeat Chart of Movies Seen by Key Demographics

Recreate this chart using .repeat()

<img src="https://scmcqueen.github.io/si649_hw/movies_demo.png" height="300">

In [66]:
# read in the formatted data
x = pd.read_csv('https://scmcqueen.github.io/si649_hw/movies_demo.csv',index_col=0)
x.head()

,Age,Gender,Do you consider yourself to be a fan of the Star Wars film franchise?,movies_seen
2,18-29,Male,No,3
3,18-29,Male,Yes,6
4,18-29,Male,Yes,6
5,18-29,Male,Yes,6
8,18-29,Male,Yes,6


In [ ]:
alt.Chart(x).mark_boxplot(extent='min-max', box=alt.MarkConfig(fill="#b62321", stroke="#b62321")).encode(
    x=alt.X(alt.repeat("column"), type='nominal', axis=alt.Axis(labelAngle=0,ticks=True)),
    y=alt.Y("movies_seen:Q", title=None, 
            scale=alt.Scale(domain=[0, 6])),
).properties(
    width=200, height=300
).repeat(
    column=[
        "Age",
        "Gender",
        "Do you consider yourself to be a fan of the Star Wars film franchise?"
    ]
).properties(
    title="Number of Movies Seen by Key Demographics"
)

alt.RepeatChart(...)

## Part 4: Make your own theme!

Make your own theme. This is your opportunity to be creative and define a consistent, reusable style. The theme must be significantly different than the Star Wars one above. The more unique, the better!

To get full points, you must:
* Create at least 3 color schemes for your theme
* Choose a new font for your theme
* Change the default font sizes
* Set a new default mark color
* Choose to enable or disable grid lines
* Change the default padding of the titles

You can choose one of the themes from [this list of Data Visualization Style Guidelines](https://docs.google.com/spreadsheets/d/1F1gm5QLXh3USC8ZFx_M9TXYxmD-X5JLDD0oJATRTuIE/edit?gid=1679646668#gid=1679646668) as a starting point. To generate interesting color palettes, you can use sites like [Coolors](https://coolors.co/), [Canva](https://www.canva.com/colors/color-palette-generator/), or [ColorMagic](https://colormagic.app/).

In [103]:
import altair as alt

def new_theme():
    font = "Roboto Slab"
    backgroundColor = "#F9F9F5"

    return {
        "config": {
            "title": {
                "font": font,
                "fontSize": 24,      
                "color": "#1A3C40",   
                "offset": 20         
            },
            "subtitle": {
                "font": font,
                "fontSize": 18,
                "color": "#1A3C40",
                "offset": 15        
            },

            "axis": {
                "labelFont": font,
                "labelFontSize": 14,
                "labelColor": "#1A3C40",
                "titleFont": font,
                "titleFontSize": 16,
                "titleColor": "#1A3C40"
            },
            "axisX": {
                "grid": False
            },
            "axisY": {
                "grid": True,
                "gridColor": "#D2D7D3",
                "gridWidth": 1
            },

            "legend": {
                "labelFont": font,
                "labelFontSize": 14,
                "titleFont": font,
                "titleFontSize": 16,
                "titleColor": "#1A3C40"
            },
            "mark": {
                "color": "#A2323E"
            },
            "range": {
                "category": [
                    "#586F6B",
                    "#9DBF9E",
                    "#D8DBB2",
                    "#E9B872",
                    "#CA7E8D",
                    "#B5515D"
                ],
                "ordinal": [
                    "#FCE1CE",
                    "#F8B6A3",
                    "#F18578",
                    "#DE5858",
                    "#B02339"
                ],
                "ramp": [
                    "#F0E5CF",
                    "#E2D19B",
                    "#C3B177",
                    "#988753",
                    "#6B5F34"
                ],
                "diverging": [
                    "#404E5C", 
                    "#7A9E9F", 
                    "#C9DBC7", 
                    "#F8F8F8",
                    "#FCDBBE", 
                    "#ED9B67", 
                    "#A0412C"
                ]
            },
            "background": backgroundColor,
            "view": {
                "stroke": "transparent"
            }
        }
    }

alt.themes.register("ljr", new_theme)
alt.themes.enable("ljr")

ThemeRegistry.enable('ljr')

### Recreate all of the charts from above in your theme!

In [111]:
# insert facet chart
bar1 = alt.Chart(df_percent).mark_bar().encode(
    x=alt.X('percent:Q', axis=alt.Axis(title=None, grid=False, labels=False, ticks=False, domain=False)),
    y=alt.Y('character:N', axis=alt.Axis(title=None, grid=False),sort=alt.EncodingSortField(field="percent_favorable", order="descending")),
    color=alt.Color('class:N', legend=None),
).properties(
    width=80,
)
text1 = bar1.mark_text(
    align='left',
    baseline='middle',
    dx=3,
    fontSize=10,
).encode(
    text=alt.Text('percent:Q', format=".0f")
)
theme_1 = (bar1 + text1).facet(
    column=alt.Column('class:N', sort=['Favorable', 'Neutral', 'Unfavorable', 'Unfamiliar'],header=alt.Header(title=None)),
).configure_view(
    stroke=None
).properties(
    title={
        "text": "'Star Wars' Favorability Rating",
        "subtitle": "By 834 respondents",
        "anchor": "start"
    }
)
theme_1


alt.FacetChart(...)

In [106]:
# insert scatter plot
alt.Chart(df_c1).mark_point(filled=True, size=50).encode(
    x=alt.X('total:Q', scale=alt.Scale(domain=[800, 850]), axis=alt.Axis(title='Number of Ratings')),
    y=alt.Y('c1:Q', axis=alt.Axis(title='Percent of Favorable Ratings', tickCount=6)),
    color=alt.Color('c1:Q', title='Percent'),
).properties(
    title='Favorable Ratings by Number of Ratings',
    width=300,
    height=300
)

alt.Chart(...)

In [108]:
bar = (
    alt.Chart(df_count)
    .mark_bar(height=30)
    .encode(
        y=alt.Y('shooter:N', sort=None, title='', axis=alt.Axis(grid=False, ticks=False)),
        x=alt.X('percent:Q',
                title=' ',
                axis=alt.Axis(title=None, grid=False, labels=False, ticks=False, domain=False),),
        color=alt.Color('percent:Q', title='Percent of Responders',
                        scale=alt.Scale(domain=[0.2, 0.3, 0.4] ))
    ).properties(
        title='Who shot first?',
        width=300,
        height=100
    )
)
text = bar.mark_text(
    align='right',
    baseline='middle',
    dx=-3,
    color='white'
).encode(
    text=alt.Text('percent:Q', format='.2%'),
    color=alt.value('white')
)

final_chart = (bar + text)
final_chart


alt.LayerChart(...)

In [110]:
alt.Chart(x).mark_boxplot(extent='min-max').encode(
    x=alt.X(alt.repeat("column"), type='nominal', axis=alt.Axis(labelAngle=0,ticks=True)),
    y=alt.Y("movies_seen:Q", title=None, 
            scale=alt.Scale(domain=[0, 6])),
).properties(
    width=200, height=300
).repeat(
    column=[
        "Age",
        "Gender",
        "Do you consider yourself to be a fan of the Star Wars film franchise?"
    ]
).properties(
    title="Number of Movies Seen by Key Demographics"
)

alt.RepeatChart(...)

## Part 5: Upload one of your charts to a github page

Choose one of the four charts above (in your own style) and export it as an html, using `chart.save('alt_theme.html')`.

Open the html file using any text editor, and add a link to the data you used for this chart, by adding a line of HTML to the body (i.e., between the `<body>` and `</body>` tags); something like `<a href="link">Data</a>`, where "link" is the URL for the data. This link to the data should appear on your html page.

Using the same process as we used for the matplotlib assignment, upload your html page to github, such that it is accessible on github pages, and insert the link to your page below:


TODO: ```Insert your link here```

Finally, open the link, and verify that you are able to download the data using the link that you added. 

Next, click on the three dots at the upper right of the plot, and click on "View Source". Note that this plot thus supports some basic interaction, even though this is a static html page. 

After clicking "View Source", copy the plot's source description and paste it below:


TODO: ```Paste the plot source here```

In [112]:
theme_1.save('alt_theme.html')